In [2]:
import pandas as pd

# Load the CSV file
file_path = 'trialMFdata - all_data.csv'
df = pd.read_csv(file_path)

# Convert 'Date' to datetime format, 'Quantity' to int, and 'Percentage Traded' to a float
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Quantity'] = df['Quantity'].str.replace(',', '').astype(int)
df['Percentage Traded'] = df['Percentage Traded'].str.replace('%', '').replace('-','0').astype(float) / 100
# Split the 'Stock' column into two at the newline character
df[['Stock', 'Tags']] = df['Stock'].str.split('\n', 1, expand=True)

# Now 'Stock' contains everything before '\n' and 'Tags' contains everything after '\n'
# If there was no '\n', 'Tags' will contain None

# If you want to fill NaN values in 'Tags' with an empty string:
df['Tags'] = df['Tags'].fillna('')

# Sort the data by Date (Latest first)
df_sorted = df.sort_values(by='Date', ascending=False)


In [3]:
df_sorted

,Stock,Institution Name,Client Name,Exchange,Deal type,Action,Date,Avg Price,Quantity,Percentage Traded,Tags
100,Data Patterns (India) Ltd.,HDFC Group,HDFC MUTUAL FUND,BSE,Block,Purchase,2024-02-16,1837.00,138000,0.0025,High Gain High Volume
602,Data Patterns (India) Ltd.,Kotak Group,KOTAK MAHINDRA MUTUAL FUND,BSE,Block,Purchase,2024-02-16,1837.00,91000,0.0016,High Gain High Volume
601,Data Patterns (India) Ltd.,Kotak Group,KOTAK MAHINDRA MUTUAL FUND,BSE,Block,Purchase,2024-02-16,1837.00,455000,0.0081,High Gain High Volume
600,Data Patterns (India) Ltd.,Kotak Group,KOTAK MAHINDRA MUTUAL FUND,BSE,Bulk,Purchase,2024-02-16,1837.00,455000,0.0081,High Gain High Volume
101,Sudarshan Chemical Industries Ltd.,HDFC Group,HDFC MUTUAL FUND,BSE,Bulk,Purchase,2024-02-15,513.99,1775551,0.0256,52 week high Insider Trades
...,...,...,...,...,...,...,...,...,...,...,...
1128,Cantabil Retail India Ltd.,Axis Group,AXIS BANK LTD.,NSE,Bulk,Purchase,2010-10-12,114.78,110000,0.0013,Dividend
1267,Consolidated Construction Consortium Ltd.,Franklin India Group,FRANKLIN TEMPLETON MUTUAL FUND,NSE,Bulk,Purchase,2010-10-06,83.00,4000000,0.0100,
1130,AK Capital Services Ltd.,Axis Group,AXIS MUTUAL FUND AC AXIS EQUITY FUND,BSE,Block,Purchase,2010-09-24,485.00,235000,0.0356,Dividend
1268,Everonn Education Ltd.,Franklin India Group,FRANKLIN TEMPLETON MUTUAL FUND,NSE,Bulk,Sell,2010-08-03,527.79,125000,0.0052,


In [4]:

# Group by Date, Stock, and Action, then aggregate
df_grouped = df_sorted.groupby(['Date', 'Stock', 'Action', 'Institution Name'], as_index=False).agg({
    'Client Name': lambda x: ', '.join(x.unique()),
    'Exchange': 'first',
    'Deal type': 'first',
    'Avg Price': 'max',
    'Quantity': 'sum',
    'Percentage Traded': 'sum'
})

In [5]:
df_purchase = df_grouped[df_grouped['Action']=="Purchase"]
df_sell = df_grouped[df_grouped['Action']=="Sell"]

In [6]:
df_purchase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 0 to 1026
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               606 non-null    datetime64[ns]
 1   Stock              606 non-null    object        
 2   Action             606 non-null    object        
 3   Institution Name   606 non-null    object        
 4   Client Name        606 non-null    object        
 5   Exchange           606 non-null    object        
 6   Deal type          606 non-null    object        
 7   Avg Price          606 non-null    float64       
 8   Quantity           606 non-null    int32         
 9   Percentage Traded  606 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(6)
memory usage: 49.7+ KB


In [7]:
df_purchase.to_csv("Step1-Purchase.csv")
df_sell.to_csv("Step1-Sell.csv")

In [8]:
import pandas as pd

df_purchase = pd.read_csv("Step1-Purchase.csv")

In [9]:
df_purchase['Exchange'].value_counts()

NSE    357
BSE    249
Name: Exchange, dtype: int64